In [4]:
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:

data = {
    'Japanese Sentence': [
        "この製品は動かない！早急に対処してください。",
        "配達が遅すぎる！サービスの質を改善してください。",
        "サポートへの連絡がつながらない。改善してください。",
        "予約した商品が売り切れだった。通知システムを改善してほしい。",
        "注文した商品と異なるものが届いた。返品手続きを教えてください。",
        "商品の品質が悪い。返金してほしい。",
        "このアプリは使いづらい。改善をお願いします。",
        "店員の態度がひどい。トレーニングを強化してください。",
        "騒音がひどくて眠れない。対策を講じてください。",
        "請求書の金額が間違っている。修正してください。",
        "商品の在庫状況を教えてください。",
        "このサービスの料金体系を詳しく教えてください。",
        "予約のキャンセル方法を知りたいです。",
        "修理サービスの利用方法を教えてください。",
        "このイベントの日程と場所を教えてください。",
        "メニューにアレルギー情報はありますか？",
        "ホテルのチェックインとチェックアウトの時間を確認したいです。",
        "オンライン注文の追跡方法を教えてください。",
        "旅行パッケージに含まれるものを教えてください。",
        "製品の保証期間はどれくらいですか？",
        "お祝いのメッセージを送ります！おめでとう！",
        "明日の会議を確認しました。参加します。",
        "新商品のお知らせです。詳細はウェブサイトでご覧ください。",
        "今週末のイベントにご招待いただき、ありがとうございます。",
        "お会いできるのを楽しみにしています！",
        "天気予報によると明日は晴れるそうですね。",
        "プロジェクトの進捗状況を報告します。完成まであと少しです。",
        "本日は休業日です。ご了承ください。",
        "お客様の声を大切にしています。アンケートにご協力ください。",
        "ありがとうございました。またのご利用をお待ちしています。"
    ],
    'Category': [
        'Complaint', 'Complaint', 'Complaint', 'Complaint', 'Complaint',
        'Complaint', 'Complaint', 'Complaint', 'Complaint', 'Complaint',
        'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry',
        'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry',
        'Others', 'Others', 'Others', 'Others', 'Others',
        'Others', 'Others', 'Others', 'Others', 'Others'
    ]
}

df = pd.DataFrame(data)

print(df)


                  Japanese Sentence   Category
0            この製品は動かない！早急に対処してください。  Complaint
1          配達が遅すぎる！サービスの質を改善してください。  Complaint
2         サポートへの連絡がつながらない。改善してください。  Complaint
3    予約した商品が売り切れだった。通知システムを改善してほしい。  Complaint
4   注文した商品と異なるものが届いた。返品手続きを教えてください。  Complaint
5                 商品の品質が悪い。返金してほしい。  Complaint
6            このアプリは使いづらい。改善をお願いします。  Complaint
7        店員の態度がひどい。トレーニングを強化してください。  Complaint
8           騒音がひどくて眠れない。対策を講じてください。  Complaint
9           請求書の金額が間違っている。修正してください。  Complaint
10                 商品の在庫状況を教えてください。    Inquiry
11          このサービスの料金体系を詳しく教えてください。    Inquiry
12               予約のキャンセル方法を知りたいです。    Inquiry
13             修理サービスの利用方法を教えてください。    Inquiry
14            このイベントの日程と場所を教えてください。    Inquiry
15              メニューにアレルギー情報はありますか？    Inquiry
16   ホテルのチェックインとチェックアウトの時間を確認したいです。    Inquiry
17            オンライン注文の追跡方法を教えてください。    Inquiry
18          旅行パッケージに含まれるものを教えてください。    Inquiry
19                製品の保証期間はどれくらいですか？    Inquiry
20           

In [11]:
lr = 1e-3
seq_len = 20
dropout = 0.5

#label_col = "Product"
tokens_path = "Output/tokens(jap).pkl"
labels_path = "Output/labels(jap).pkl"
model_path = "Output/bert_pre_trained(jap).pth"
#text_col_name = "Consumer complaint narrative"
label_encoder_path = "Output/label_encoder(jap).pkl"


In [8]:
def save_file(name, obj):
    """
    Function to save an object as pickle file
    """
    with open(name, 'wb') as f:
        pickle.dump(obj, f)


def load_file(name):
    """
    Function to load a pickle object
    """
    return pickle.load(open(name, "rb"))

## Process text data
---

In [9]:
data = df

In [10]:
df

,Japanese Sentence,Category
0,この製品は動かない！早急に対処してください。,Complaint
1,配達が遅すぎる！サービスの質を改善してください。,Complaint
2,サポートへの連絡がつながらない。改善してください。,Complaint
3,予約した商品が売り切れだった。通知システムを改善してほしい。,Complaint
4,注文した商品と異なるものが届いた。返品手続きを教えてください。,Complaint
5,商品の品質が悪い。返金してほしい。,Complaint
6,このアプリは使いづらい。改善をお願いします。,Complaint
7,店員の態度がひどい。トレーニングを強化してください。,Complaint
8,騒音がひどくて眠れない。対策を講じてください。,Complaint
9,請求書の金額が間違っている。修正してください。,Complaint


In [5]:
#data.dropna(subset=[text_col_name], inplace=True)

In [6]:
#data=data.iloc[:10000]

In [7]:
#data.replace({label_col: product_map}, inplace=True)

### Encode labels

In [12]:
label_encoder = LabelEncoder()
label_encoder.fit(data['Category'])
labels = label_encoder.transform(data['Category'])

In [13]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
save_file(labels_path, labels)
save_file(label_encoder_path, label_encoder)

### Process the text column

In [15]:
input_text = list(data["Japanese Sentence"])

In [16]:
len(input_text)

30

### Convert text to lower case

In [14]:
#input_text = [i.lower() for i in tqdm(input_text)]

100%|████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 411989.86it/s]


### Remove punctuations except apostrophe

re.sub(r"[^\w\d'\s]+", " ", i)는 re 모듈의 sub 함수를 사용하여 문자열 i에서 [^\w\d'\s]+ 패턴에 매치되는 모든 부분을 공백으로 대체합니다.

여기서 정규 표현식 [^\w\d'\s]+은 다음과 같이 구성됩니다:

[^\w\d'\s]: \w는 알파벳과 숫자를 의미하며, \d는 숫자를 의미합니다. '\s'는 공백 문자를 의미하고, [^...]는 해당 문자들을 제외한 모든 문자를 의미합니다.

In [17]:
input_text = [re.sub(r"[^\w\d'\s]+", " ", i)
             for i in tqdm(input_text)]

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 5697.75it/s]


In [18]:
input_text

['この製品は動かない 早急に対処してください ',
 '配達が遅すぎる サービスの質を改善してください ',
 'サポートへの連絡がつながらない 改善してください ',
 '予約した商品が売り切れだった 通知システムを改善してほしい ',
 '注文した商品と異なるものが届いた 返品手続きを教えてください ',
 '商品の品質が悪い 返金してほしい ',
 'このアプリは使いづらい 改善をお願いします ',
 '店員の態度がひどい トレーニングを強化してください ',
 '騒音がひどくて眠れない 対策を講じてください ',
 '請求書の金額が間違っている 修正してください ',
 '商品の在庫状況を教えてください ',
 'このサービスの料金体系を詳しく教えてください ',
 '予約のキャンセル方法を知りたいです ',
 '修理サービスの利用方法を教えてください ',
 'このイベントの日程と場所を教えてください ',
 'メニューにアレルギー情報はありますか ',
 'ホテルのチェックインとチェックアウトの時間を確認したいです ',
 'オンライン注文の追跡方法を教えてください ',
 '旅行パッケージに含まれるものを教えてください ',
 '製品の保証期間はどれくらいですか ',
 'お祝いのメッセージを送ります おめでとう ',
 '明日の会議を確認しました 参加します ',
 '新商品のお知らせです 詳細はウェブサイトでご覧ください ',
 '今週末のイベントにご招待いただき ありがとうございます ',
 'お会いできるのを楽しみにしています ',
 '天気予報によると明日は晴れるそうですね ',
 'プロジェクトの進捗状況を報告します 完成まであと少しです ',
 '本日は休業日です ご了承ください ',
 'お客様の声を大切にしています アンケートにご協力ください ',
 'ありがとうございました またのご利用をお待ちしています ']

### Remove digits

In [19]:
input_text = [re.sub("\d+", "", i) for i in tqdm(input_text)]

100%|███████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 27752.34it/s]


In [20]:
input_text

['この製品は動かない 早急に対処してください ',
 '配達が遅すぎる サービスの質を改善してください ',
 'サポートへの連絡がつながらない 改善してください ',
 '予約した商品が売り切れだった 通知システムを改善してほしい ',
 '注文した商品と異なるものが届いた 返品手続きを教えてください ',
 '商品の品質が悪い 返金してほしい ',
 'このアプリは使いづらい 改善をお願いします ',
 '店員の態度がひどい トレーニングを強化してください ',
 '騒音がひどくて眠れない 対策を講じてください ',
 '請求書の金額が間違っている 修正してください ',
 '商品の在庫状況を教えてください ',
 'このサービスの料金体系を詳しく教えてください ',
 '予約のキャンセル方法を知りたいです ',
 '修理サービスの利用方法を教えてください ',
 'このイベントの日程と場所を教えてください ',
 'メニューにアレルギー情報はありますか ',
 'ホテルのチェックインとチェックアウトの時間を確認したいです ',
 'オンライン注文の追跡方法を教えてください ',
 '旅行パッケージに含まれるものを教えてください ',
 '製品の保証期間はどれくらいですか ',
 'お祝いのメッセージを送ります おめでとう ',
 '明日の会議を確認しました 参加します ',
 '新商品のお知らせです 詳細はウェブサイトでご覧ください ',
 '今週末のイベントにご招待いただき ありがとうございます ',
 'お会いできるのを楽しみにしています ',
 '天気予報によると明日は晴れるそうですね ',
 'プロジェクトの進捗状況を報告します 完成まであと少しです ',
 '本日は休業日です ご了承ください ',
 'お客様の声を大切にしています アンケートにご協力ください ',
 'ありがとうございました またのご利用をお待ちしています ']

### Remove more than one consecutive instance of 'x'

In [21]:
input_text = [re.sub(r'[x]{2,}', "", i) for i in tqdm(input_text)]

100%|██████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<?, ?it/s]


### Remove multiple spaces with single space

In [22]:
input_text = [re.sub(' +', ' ', i) for i in tqdm(input_text)]

100%|██████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<?, ?it/s]


In [23]:
input_text

['この製品は動かない 早急に対処してください ',
 '配達が遅すぎる サービスの質を改善してください ',
 'サポートへの連絡がつながらない 改善してください ',
 '予約した商品が売り切れだった 通知システムを改善してほしい ',
 '注文した商品と異なるものが届いた 返品手続きを教えてください ',
 '商品の品質が悪い 返金してほしい ',
 'このアプリは使いづらい 改善をお願いします ',
 '店員の態度がひどい トレーニングを強化してください ',
 '騒音がひどくて眠れない 対策を講じてください ',
 '請求書の金額が間違っている 修正してください ',
 '商品の在庫状況を教えてください ',
 'このサービスの料金体系を詳しく教えてください ',
 '予約のキャンセル方法を知りたいです ',
 '修理サービスの利用方法を教えてください ',
 'このイベントの日程と場所を教えてください ',
 'メニューにアレルギー情報はありますか ',
 'ホテルのチェックインとチェックアウトの時間を確認したいです ',
 'オンライン注文の追跡方法を教えてください ',
 '旅行パッケージに含まれるものを教えてください ',
 '製品の保証期間はどれくらいですか ',
 'お祝いのメッセージを送ります おめでとう ',
 '明日の会議を確認しました 参加します ',
 '新商品のお知らせです 詳細はウェブサイトでご覧ください ',
 '今週末のイベントにご招待いただき ありがとうございます ',
 'お会いできるのを楽しみにしています ',
 '天気予報によると明日は晴れるそうですね ',
 'プロジェクトの進捗状況を報告します 完成まであと少しです ',
 '本日は休業日です ご了承ください ',
 'お客様の声を大切にしています アンケートにご協力ください ',
 'ありがとうございました またのご利用をお待ちしています ']

### Tokenize the text

In [24]:
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/104 [00:00<?, ?B/s]

In [25]:
input_text[0]

'この製品は動かない 早急に対処してください '

In [26]:
sample_tokens = tokenizer(input_text[0], padding="max_length",
                         max_length=seq_len, truncation=True,
                         return_tensors="pt")

In [27]:
sample_tokens

{'input_ids': tensor([[    2,    70,   338,  1227,     9,   531, 12580, 28457,  1214,  1132,
             7,   216,  1106,    15, 28456, 28504, 28565,  7103,     3,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [28]:
sample_tokens["input_ids"]

tensor([[    2,    70,   338,  1227,     9,   531, 12580, 28457,  1214,  1132,
             7,   216,  1106,    15, 28456, 28504, 28565,  7103,     3,     0]])

In [29]:
sample_tokens["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])

In [30]:
tokens = [tokenizer(i, padding="max_length", max_length=seq_len, 
                    truncation=True, return_tensors="pt") 
         for i in tqdm(input_text)]

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 1816.69it/s]


In [31]:
tokens

[{'input_ids': tensor([[    2,    70,   338,  1227,     9,   531, 12580, 28457,  1214,  1132,
              7,   216,  1106,    15, 28456, 28504, 28565,  7103,     3,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])},
 {'input_ids': tensor([[    2,   549,   784,    14,  2333,  8896,  1645, 28444,  2015,    11,
            307,  3743,    15, 28456, 28504, 28565,  7103,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])},
 {'input_ids': tensor([[    2,  4283, 28647, 28444,   195,  8145,    14, 28532,  4319,  3721,
            307,  3743,    15, 28456, 28504, 28565,  7103,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask':

### Save the tokens

In [32]:
save_file(tokens_path, tokens)

## Create Bert model
---

In [63]:
class BertForClassification(nn.Module):
    def __init__(self, num_classes):
        super(BertForClassification, self).__init__()
        self.bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

In [64]:
train_texts, test_texts, train_labels, test_labels = train_test_split(input_text, labels, test_size=0.2, random_state=42)

In [65]:
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt', max_length=seq_len)
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt', max_length=seq_len)

In [66]:
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

In [67]:
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [68]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

In [69]:
model = BertForClassification(num_classes=len(label_encoder.classes_))

In [70]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [90]:
from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted_labels = torch.max(outputs, dim=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)
        
        progress_bar.set_postfix(loss=loss.item(), refresh=True)
    
    train_loss = total_loss / len(train_loader)
    train_accuracy = correct_predictions / total_predictions
    
    # Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        correct_predictions = 0
        total_predictions = 0
        
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted_labels = torch.max(outputs, dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)
        
        val_accuracy = correct_predictions / total_predictions
    
    progress_bar.set_postfix(loss=train_loss, train_acc=train_accuracy, val_acc=val_accuracy)
    progress_bar.close()  # Close the progress bar for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f} - "
          f"Train Accuracy: {train_accuracy:.4f} - "
          f"Validation Accuracy: {val_accuracy:.4f}")



Epoch [1/10]: 100%|██████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.51s/it, loss=0.741]


Epoch [1/10] - Train Loss: 0.7305 - Train Accuracy: 0.9583 - Validation Accuracy: 0.3333


Epoch [2/10]: 100%|██████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.38s/it, loss=0.636]


Epoch [2/10] - Train Loss: 0.6811 - Train Accuracy: 1.0000 - Validation Accuracy: 0.3333


Epoch [3/10]: 100%|████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.38s/it, loss=0.7]


...

Epoch [8/10] - Train Loss: 0.4384 - Train Accuracy: 1.0000 - Validation Accuracy: 0.6667


Epoch [9/10]: 100%|██████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.37s/it, loss=0.466]


Epoch [9/10] - Train Loss: 0.4390 - Train Accuracy: 1.0000 - Validation Accuracy: 0.6667


Epoch [10/10]: 100%|█████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.35s/it, loss=0.371]


Epoch [10/10] - Train Loss: 0.3773 - Train Accuracy: 1.0000 - Validation Accuracy: 0.8333


In [86]:
torch.save(model.state_dict(), model_path)
print("Model saved!")

Model saved!


In [87]:
model.load_state_dict(torch.load(model_path))
model.eval()

BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [88]:
def predict(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=seq_len)
    with torch.no_grad():
        outputs = model(encodings['input_ids'], attention_mask=encodings['attention_mask'])
        _, predicted_labels = torch.max(outputs, dim=1)
        predicted_labels = predicted_labels.cpu().numpy()
        predicted_classes = label_encoder.inverse_transform(predicted_labels)
        return predicted_classes

In [89]:
example_texts = ["製品の不具合について問い合わせです。", "イベントの詳細を教えてください。"]
predicted_classes = predict(example_texts)
print("Predicted Classes:", predicted_classes)

Predicted Classes: ['Inquiry' 'Inquiry']
